# **Tidying**

Part of the data engineering process is data cleaning and tidying. What is done in those two processes, is trying to make the data more readable, and complete. This makes much easier to analyze, visualize, and train the data.


## **Data Tidying**

Making the data more organized, and readable is the result of applying data tidying. 

In this section two main pandas functions are used in data tidying those are `melt` and `pivot_table`.

Let's start by taking a look at the below dataframe, which represents the income ranges based on religion. This is part of the PEW research, which is famous in the US for conducting pollings and surveys on citizens.

When the following are satisfied:


1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

We can then say that our dataset is *tidy*.

First we need to import pandas to read csv datasets.

In [10]:
!pip install pandas
import pandas as pd
import numpy as np


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### **PEW Research Dataset**

1. Start by Importing the dataset into a pandas dataframe.

In [11]:
df_pew = pd.read_csv('./data/pew-raw.csv')


2. Observe the dataset using the `loc`, `iloc`, `head`, or `tail` approaches

In [12]:
# Display the 'religion' and '<$10k' columns using loc
df_pew.columns = df_pew.columns.str.strip().str.lower()
df_pew.loc[:, ['religion', '<$10k']]

,religion,<$10k
0,Agnostic,27
1,Atheist,12
2,Buddhist,27
3,Catholic,418
4,Dont know/refused,15
5,Evangelical Prot,575
6,Hindu,1
7,Historically Black Prot,228
8,Jehovahs Witness,20
9,Jewish,19


In [13]:
# Display the first 5 rows of the dataframe
df_pew.head()


,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35


In [14]:
# Display the last 5 rows of the dataframe
df_pew.tail()


,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


In [15]:
# Display the first 3 rows using iloc
df_pew.iloc[:3]

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58


***What does not seem right in the above dataframe?***

3. Try to make the column headers represent a variable not a value. For that, use the `melt` function.

In [16]:
df_pew_melted = pd.melt(df_pew, id_vars=['religion'], var_name='income', value_name='count')
df_pew_melted.head()

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Dont know/refused,<$10k,15


## Using Replace function

In [17]:
# Replace the income range values in the 'income' column for better readability
df_pew_melted['income'] = df_pew_melted['income'].replace({
    '<$10k': 'Less than $10k',
    '$10-20k': '$10k-$20k',
    '$20-30k': '$20k-$30k',
    '$30-40k': '$30k-$40k',
    '$40-50k': '$40k-$50k',
    '$50-75k': '$50k-$75k'
})
df_pew_melted.head()

,religion,income,count
0,Agnostic,Less than $10k,27
1,Atheist,Less than $10k,12
2,Buddhist,Less than $10k,27
3,Catholic,Less than $10k,418
4,Dont know/refused,Less than $10k,15


In [18]:
# Replace '$' and 'k' with blank and 'Less than 10k' with '0-10' in the 'income' column
df_pew_melted['income'] = df_pew_melted['income'].str.replace('$', '', regex=False)
df_pew_melted['income'] = df_pew_melted['income'].str.replace('k', '', regex=False)
df_pew_melted['income'] = df_pew_melted['income'].replace({'Less than 10': '0-10'})
df_pew_melted['income'] = df_pew_melted['income'].str.replace('$', '', regex=False)
df_pew_melted['income'] = df_pew_melted['income'].replace({'Less than 10k': '0-10k'})
df_pew_melted.head()

,religion,income,count
0,Agnostic,0-10,27
1,Atheist,0-10,12
2,Buddhist,0-10,27
3,Catholic,0-10,418
4,Dont know/refused,0-10,15


In [19]:
# Split the 'income' column into 'income_min' and 'income_max'
df_pew_melted[['income_min', 'income_max']] = df_pew_melted['income'].str.split('-', expand=True)

# Display the updated dataframe
df_pew_melted.head()

,religion,income,count,income_min,income_max
0,Agnostic,0-10,27,0,10
1,Atheist,0-10,12,0,10
2,Buddhist,0-10,27,0,10
3,Catholic,0-10,418,0,10
4,Dont know/refused,0-10,15,0,10


### **Billboard Dataset**

This dataset outlines data about the top hit songs on the Billboard list and the week from entrance that it was in the billboard with the ranking.

1. Read the dataset and store it in a pandas dataframe. Note that the usual utf-8 encoding does not work on this dataset. The reason behind this is that there might be characters that are not supported by `utf-8`.

The suggestion is to use for this dataset `unicode_escape` encoding. (converts all non-ASCII characters into their \uXXXX representations)

2. Observe the first few rows of the dataset.

***What is wrong with the above dataset?***

3. Let's, again, use the `melt` function to fix the general structure of the dataframe.

If we inspect the current dataframe. We find that it is structured in a better way than before. 

However, the ***Week*** column looks a bit ugly!

4. Let's try to place only the week number in that column without the extras surronding it.

5. Now let's inspect the ***Week*** column in the dataframe.

Next, let's try to find the date at which the song ranked the number that is shown per row.

6. To do that let's first think of the equation that is going to get us the relevant date at which the song ranked the *rth*.



*Timedeltas are absolute differences in times, expressed in difference units (e.g. days, hours, minutes, seconds). This method converts an argument from a recognized timedelta format / value into a Timedelta type.*


***What is the problem with the calculation above?***

7. Let's only keep necessary columns

8. How to rename your columns?

Display the dataframe

In the above dataframe, there are some *NaN* values. What are we going to do? <br/>
9. Apply quick data cleaning and then observe the dataset

## **Data Cleaning**

Data cleaning involves removing unwanted characters, imputing, or dropping missing values.

The decision is based on the dataset you have, and the information you can extract from the other columns.


Examples of data cleaning include cleaning:

1.   **Missing Data**
2.   **Irregular Data** (Outliers)
3.   **Unnecessary Data** — Repetitive Data, Duplicates and more
4.   **Inconsistent Data** — Capitalization, Addresses and more








### **Cars Data Set**

Start by reading the dataset related to car models: ./CSVs/cars.csv

In [7]:
from io import StringIO
import pandas as pd

# Reading the file from Data folder


# Optional: Adjust display settings for better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Display the DataFrame
print(df_cars)


                           Car     MPG Cylinders Displacement Horsepower  Weight Acceleration Model  Origin
0                       STRING  DOUBLE       INT       DOUBLE     DOUBLE  DOUBLE       DOUBLE   INT     CAT
1    Chevrolet Chevelle Malibu     NaN         8        307.0      130.0   3504.         12.0    70      US
2            Buick Skylark 320    15.0         8        350.0        NaN   3693.         11.5    70      US
3           Plymouth Satellite     NaN         8        318.0      150.0   3436.         11.0    70      US
4                AMC Rebel SST    16.0         8          NaN      150.0     NaN         12.0    70      US
..                         ...     ...       ...          ...        ...     ...          ...   ...     ...
402            Ford Mustang GL    27.0         4        140.0      86.00   2790.         15.6    82      US
403          Volkswagen Pickup    44.0         4        97.00      52.00   2130.         24.6    82  Europe
404              Dodge Rampa

First Row seems to be the datatype, we need to remove it

In [8]:
# Remove the first row which contains the datatype information
df_cars = df_cars.iloc[1:].reset_index(drop=True)
df_cars.head()

,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
0,Chevrolet Chevelle Malibu,NaN,8,307.0,130.0,3504.,12.0,70,US
1,Buick Skylark 320,15.0,8,350.0,NaN,3693.,11.5,70,US
2,Plymouth Satellite,NaN,8,318.0,150.0,3436.,11.0,70,US
3,AMC Rebel SST,16.0,8,NaN,150.0,NaN,12.0,70,US
4,Ford Torino,17.0,8,302.0,140.0,3449.,10.5,70,US


Let's observe the columns with null values.  Either by using the `isnull().sum()` function 

In [9]:
df_cars.isnull().sum()

Car             0
MPG             3
Cylinders       0
Displacement    1
Horsepower      2
Weight          1
Acceleration    0
Model           0
Origin          0
dtype: int64

There aren'at many missing values. Let's take a glimpse at the percentage of the missing values:

**HINT:** We'll need `Numpy` for the below task.

In [10]:
import numpy as np

# Calculate the percentage of missing values for each column using numpy

missing_percent = df_cars.isnull().sum() / len(df_cars) * 100
print(missing_percent)

Car             0.000000
MPG             0.738916
Cylinders       0.000000
Displacement    0.246305
Horsepower      0.492611
Weight          0.246305
Acceleration    0.000000
Model           0.000000
Origin          0.000000
dtype: float64


Around *0.19%* of the values are missing, which isn't a lot. Therefore, we might go with the option of dropping all the rows with null values.

Lets also check dropping the columns

Let's observe how many columns we lost

In [12]:
# Drop all rows with any missing values
df_cars_dropped = df_cars.dropna().reset_index(drop=True)
# Drop columns with any missing values
df_cars_col_dropped = df_cars.dropna(axis=1)
print(f"Columns before dropping: {df_cars.shape[1]}")
print(f"Columns after dropping: {df_cars_col_dropped.shape[1]}")
print("Dropped columns:", set(df_cars.columns) - set(df_cars_col_dropped.columns))
df_cars_col_dropped.head()
# Display the shape of the new DataFrame to see how many rows remain
print(f"Rows before dropping: {len(df_cars)}")
print(f"Rows after dropping: {len(df_cars_dropped)}")
df_cars_dropped.head()

Columns before dropping: 9
Columns after dropping: 5
Dropped columns: {'Horsepower', 'MPG', 'Displacement', 'Weight'}
Rows before dropping: 406
Rows after dropping: 401


,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
0,Ford Torino,17.0,8,302.0,140.0,3449.,10.5,70,US
1,Chevrolet Impala,14.0,8,454.0,220.0,4354.,9.0,70,US
2,Plymouth Fury iii,14.0,8,440.0,215.0,4312.,8.5,70,US
3,Pontiac Catalina,14.0,8,455.0,225.0,4425.,10.0,70,US
4,AMC Ambassador DPL,15.0,8,390.0,190.0,3850.,8.5,70,US


### **Cars Dataset - Filling in missing values automatically**

Another option is to try and fill in the missing values through imputations.

In [14]:
from sklearn.impute import SimpleImputer

# Convert relevant columns to numeric for imputation
cols_to_impute = ['MPG', 'Displacement', 'Horsepower', 'Weight']
df_cars[cols_to_impute] = df_cars[cols_to_impute].apply(pd.to_numeric, errors='coerce')

# Create a SimpleImputer with strategy 'mean'
imputer = SimpleImputer(strategy='mean')

# Fit and transform the columns with missing values
df_cars[cols_to_impute] = imputer.fit_transform(df_cars[cols_to_impute])

# Display the DataFrame after imputation
df_cars.head()

,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
0,Chevrolet Chevelle Malibu,0.0,8,307.000000,130.000000,3504.000000,12.0,70,US
1,Buick Skylark 320,15.0,8,350.000000,103.143564,3693.000000,11.5,70,US
2,Plymouth Satellite,0.0,8,318.000000,150.000000,3436.000000,11.0,70,US
3,AMC Rebel SST,16.0,8,194.509877,150.000000,2978.293827,12.0,70,US
4,Ford Torino,17.0,8,302.000000,140.000000,3449.000000,10.5,70,US


Let's take the `MPG` column for example. We can fill in the missing values with 0s through the following line of code:

`df_cars.fillna(0) `. 

In [13]:
# Fill missing values in the 'MPG' column with 0
df_cars['MPG'] = df_cars['MPG'].fillna(0)
df_cars.head()

,Car,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model,Origin
0,Chevrolet Chevelle Malibu,0,8,307.0,130.0,3504.,12.0,70,US
1,Buick Skylark 320,15.0,8,350.0,NaN,3693.,11.5,70,US
2,Plymouth Satellite,0,8,318.0,150.0,3436.,11.0,70,US
3,AMC Rebel SST,16.0,8,NaN,150.0,NaN,12.0,70,US
4,Ford Torino,17.0,8,302.0,140.0,3449.,10.5,70,US


However, this does not make much sense as there isn't MPG equal to 0. How about we plot the MPG column and if it follows a random distribution we can use the mean of the column to compute the missing values. Otherwise, we can use the median (if there is a skewed normal distribution). However, there might be a better way of imputation which is getting the median or the mean of the MPG of the cars with similar attributes.

If we observe the graph above, we can consider it in a way or another normally distributed. Therefore, we can impute the missing values using the mean.

To compute the mean we need numeric values. However the values in the dataframe are objects. Therefore, we need to change them to numerics so that we can compute them.

Now let's see what is the mean of the MPG column

We can use this mean to compute the missing values since the graph demonstarted a normal distribution

### **Car Dataset - Simple Imputer**

*SimpleImputer* is a `scikit-learn` class which is helpful in handling the missing data in the predictive model dataset. It replaces the `NaN` values with a specified placeholder.
It is implemented by the use of the `SimpleImputer()` method which takes the following arguments :

`missing_values` : The missing_values placeholder which has to be imputed. By default is NaN

`strategy` : The data which will replace the NaN values from the dataset. The strategy argument can take the values – ‘mean'(default), ‘median’, ‘most_frequent’ and ‘constant’.


Let's start by importing the `SimpleImputer` into our notebook

What we need to do are two essential steps:

1. fit the data (compute the mean / median / most freq)
2. transform the data (place the computed values in the NaN cells)

## **Outlier Detection** 


An Outlier is a data-item/object that deviates significantly from the rest of the (so-called normal)objects. They can be caused by measurement or execution errors. The analysis for outlier detection is referred to as outlier mining. There are many ways to detect the outliers, and the removal process is the data frame same as removing a data item from the panda’s data frame.



https://www.geeksforgeeks.org/detect-and-remove-the-outliers-using-python/





### Outliers Visualization

#### Visualizing Outliers Using Box Plot
It captures the summary of the data effectively and efficiently with only a simple box and whiskers. Boxplot summarizes sample data using 25th, 50th, and 75th percentiles. One can just get insights(quartiles, median, and outliers) into the dataset by just looking at its boxplot.

#### Visualizing Outliers Using ScatterPlot.

It is used when you have paired numerical data and when your dependent variable has multiple values for each reading independent variable, or when trying to determine the relationship between the two variables. In the process of utilizing the scatter plot, one can also use it for outlier detection.

#### Z-Score:
Z- Score is also called a standard score. This value/score helps to understand that how far is the data point from the mean. And after setting up a threshold value one can utilize z score values of data points to define the outliers.
<br>
Zscore = (data_point -mean) / std. deviation



Now to define an outlier threshold value is chosen which is generally 3.0. As 99.7% of the data points lie between +/- 3 standard deviation (using Gaussian Distribution approach).

Rows where Z value is greater than 2

#### IQR (Inter-Quartile Range)
Inter Quartile Range approach to finding the outliers is the most commonly used and most trusted approach used in the research field. <Br>
IQR = Quartile3 - Quartile1

To define the outlier base value is defined above and below dataset’s normal range namely Upper and Lower bounds, define the upper and the lower bound (1.5*IQR value is considered) :<br>
upper = Q3 + 1.5 * IQR <br>
lower = Q1 - 1.5 * IQR <br> 

- Removing the outliers:
For removing the outlier, one must follow the same process of removing an entry from the dataset using its exact position in the dataset because in all the above methods of detecting the outliers end result is the list of all those data items that satisfy the outlier definition according to the method used.